# Setup

In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
#quiet_library(Nebulosa)
quiet_library(ArchR)

In [ ]:
addArchRThreads(32)
addArchRGenome("hg38")

In [ ]:
proj <- loadArchRProject(path = '../PedSen_ATAC/')
proj

## ATAC Clustering

In [ ]:
proj <- addIterativeLSI(proj, name = 'IterativeLSI', force = TRUE, varFeatures = 75000)

In [ ]:
proj <- addHarmony(proj, reducedDims = 'IterativeLSI', groupBy = c('batch_id'), force = TRUE)

In [ ]:
# proj <- addClusters(proj, reducedDims = 'IterativeLSI', name = 'Clusters', force = TRUE, resolution = 0.2)
proj <- addClusters(proj, reducedDims = 'Harmony', name = 'Clusters', force = TRUE, resolution = 0.2)

In [ ]:
# proj <- addUMAP(proj, reducedDims = 'IterativeLSI', name = 'UMAP', force = TRUE)
proj <- addUMAP(proj, reducedDims = 'Harmony', name = 'UMAP', force = TRUE)

In [ ]:
options(repr.plosp_mergeidth = 8, repr.plot.height = 8)
plotEmbedding(ArchRProj = proj, colorBy = "cellColData", name = "Clusters", embedding = "UMAP")
plotEmbedding(ArchRProj = proj, colorBy = "cellColData", name = "pediatric_senior", embedding = "UMAP")
plotEmbedding(ArchRProj = proj, colorBy = "cellColData", name = "gating_celltype", embedding = "UMAP")

In [ ]:
plotEmbedding(ArchRProj = proj, colorBy = "cellColData", name = "predicted.t_celltype.l3", embedding = "UMAP")

# Peak Analysis

In [ ]:
proj <- addGroupCoverages(ArchRProj = proj, groupBy = "gating_celltype", maxReplicates = 16, maxCells = 2000, force = TRUE)

In [ ]:
pathToMacs2 <- findMacs2()

In [ ]:
proj <- addReproduciblePeakSet(
                        ArchRProj = proj, 
                        groupBy = "gating_celltype", 
                        pathToMacs2 = pathToMacs2, force = TRUE
                            )

In [ ]:
proj <- addPeakMatrix(proj, force = TRUE)

In [ ]:
markersPeaks <- getMarkerFeatures(
    ArchRProj = proj, 
    useMatrix = "PeakMatrix", 
    groupBy = "gating_celltype",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  testMethod = "wilcoxon",
    maxCells = 10000
)

In [ ]:
heatmapPeaks <- markerHeatmap(
  seMarker = markersPeaks, 
  cutOff = "FDR <= 0.001 & Log2FC >= 0.5",
  transpose = TRUE,
  plotLog2FC = TRUE
)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 5)
draw(heatmapPeaks, heatmap_legend_side = "bot", annotation_legend_side = "bot")

# Motif Analysis

In [ ]:
# proj <- addMotifAnnotations(ArchRProj = proj, motifSet = "cisbp", name = "Motif", force = TRUE)
proj <- addMotifAnnotations(ArchRProj = proj, motifSet = "JASPAR2020", name = "JASPARMotif", force = TRUE)

In [ ]:
enrichMotifs <- peakAnnoEnrichment(
    seMarker = markersPeaks,
    ArchRProj = proj,
    peakAnnotation = "JASPARMotif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )

In [ ]:
heatmapEM <- plotEnrichHeatmap(enrichMotifs, n = 10, transpose = TRUE)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")

# ChromVar Motifs

In [ ]:
proj <- addBgdPeaks(proj)

In [ ]:
proj <- addDeviationsMatrix(
  ArchRProj = proj, 
  peakAnnotation = "JASPARMotif",
  force = TRUE
)

In [ ]:
proj <- addImputeWeights(proj)

In [ ]:
motifs <- c('RORA','RORB','EOMES','JUN','FOS','DNMT1','SMAD5','ZBTB7A','LEF1','TCF7','SOX4')
markerMotifs <- getFeatures(proj, select = paste(motifs, collapse="|"), useMatrix = "JASPARMotifMatrix")
markerMotifs

In [ ]:
markerMotifs <- grep("z:", markerMotifs, value = TRUE)
# markerMotifs <- markerMotifs[markerMotifs %ni% "z:SREBF1_22"]
markerMotifs

In [ ]:
plotVarDev <- getVarDeviations(proj, name = "JASPARMotifMatrix", plot = TRUE)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)
plotVarDev

## Motif UMAPs Imputed

In [ ]:
p <- plotEmbedding(
    ArchRProj = proj, 
    colorBy = "JASPARMotifMatrix", 
    name = sort(markerMotifs), 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(proj)
)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 40)
p2 <- lapply(p, function(x){
    x + guides(color = FALSE, fill = FALSE) + 
    theme_ArchR(baseSize = 6.5) +
    theme(plot.margin = unit(c(0, 0, 0, 0), "cm")) +
    theme(
        axis.text.x=element_blank(), 
        axis.ticks.x=element_blank(), 
        axis.text.y=element_blank(), 
        axis.ticks.y=element_blank()
    )
})
do.call(cowplot::plot_grid, c(list(ncol = 3),p2))

# Save

In [ ]:
saveArchRProject(ArchRProj = proj, outputDirectory = '../PedSen_ATAC/', load = F)

In [ ]:
sessionInfo()